# Conversation Buffer Memory

## Colab-specific setup

Make sure you have a Database, a Secure Connect Bundle and a Database Token. See [DB Setup](https://cassio.org/db_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.
Get ready to upload the Bundle and supply the Token string.

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly. See [API Setup](https://cassio.org/api_setup/) on cassio.org for details, **paying attention to the notes for Colab users**.

_Note: this colab is autogenerated from a regular Jupyter notebook hosted by cassio.org. Run all cells in this section to complete the setup before moving on to the demo content proper._

In [ ]:
# install required dependencies
! pip install \
    "git+https://github.com/hemidactylus/langchain@cassio#egg=langchain" \
    "git+https://github.com/datastax/python-driver.git@cep-vsearch#egg=cassandra-driver" \
    "cassio>=0.0.2" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "python-dotenv==1.0.0" \
    "tensorflow-cpu==2.12.0" \
    "tiktoken==0.4.0" \
    "transformers>=4.29.2"

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_APPLICATION_TOKEN = input('Your Astra DB Token: ')

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# Set your secret(s) for LLM access (key names must match `providerValidator` in `llm_choice`)
llmProvider = 'OpenAI'  # 'VertexAI'
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
else:
    raise ValueError('Unknown/unsupported LLM Provider')

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider

ASTRA_DB_CLIENT_ID = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_CLIENT_ID,
                ASTRA_DB_APPLICATION_TOKEN,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unknown CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unknown CQL Session mode')

### Colab preamble completed

The following cells constitute the demo notebook proper.

# Conversation Buffer Memory

The "base memory class" seen in the previous example is now put to use in a higher-level abstraction provided by LangChain: 

In [1]:
from langchain.memory import CassandraChatMessageHistory
from langchain.memory import ConversationBufferMemory

In [2]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

In [3]:
message_history = CassandraChatMessageHistory(
    session_id='conversation-0123',
    session=session,
    keyspace=keyspace,
    ttl_seconds = 3600,
)
message_history.clear()

## Use in a ConversationChain

### Create a Memory

The Cassandra message history is specified:

In [4]:
cassBuffMemory = ConversationBufferMemory(
    chat_memory=message_history,
)

### Language model

Below is the logic to instantiate the LLM of choice. We choose to leave it in the notebooks for clarity.

In [5]:
# creation of the LLM resources


if llmProvider == 'VertexAI':
    from langchain.llms import VertexAI
    llm = VertexAI()
    print('LLM from VertexAI')
elif llmProvider == 'OpenAI':
    from langchain.llms import OpenAI
    llm = OpenAI()
    print('LLM from OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

LLM from OpenAI


### Create a chain

As the conversation proceeds, a growing history of past exchanges finds it way automatically to the prompt that the LLM receives:

In [6]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=cassBuffMemory,
)

In [7]:
conversation.predict(input="Hello, how can I roast an apple?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, how can I roast an apple?
AI:

> Finished chain.


' Hi there! Roasting an apple is a great way to bring out a delicious flavor. To roast an apple, you will need an oven-safe dish, a few tablespoons of butter, and some spices of your choice. Preheat the oven to 375 degrees Fahrenheit and place the apple in the dish. Cut a few slits in the top of the apple, and then add the butter and spices. Bake the apple in the oven for about 20 minutes or until the skin begins to brown. Enjoy!'

In [8]:
conversation.predict(input="Can I do it on a bonfire?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, how can I roast an apple?
AI:  Hi there! Roasting an apple is a great way to bring out a delicious flavor. To roast an apple, you will need an oven-safe dish, a few tablespoons of butter, and some spices of your choice. Preheat the oven to 375 degrees Fahrenheit and place the apple in the dish. Cut a few slits in the top of the apple, and then add the butter and spices. Bake the apple in the oven for about 20 minutes or until the skin begins to brown. Enjoy!
Human: Can I do it on a bonfire?
AI:

> Finished chain.


' Unfortunately, roasting an apple on a bonfire is not recommended. While it is possible to do, it is much more difficult to maintain an even temperature and you may end up burning the apple or not roasting it enough. It is much easier to roast an apple in an oven.'

In [9]:
conversation.predict(input="What about a microwave, would the apple taste good?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, how can I roast an apple?
AI:  Hi there! Roasting an apple is a great way to bring out a delicious flavor. To roast an apple, you will need an oven-safe dish, a few tablespoons of butter, and some spices of your choice. Preheat the oven to 375 degrees Fahrenheit and place the apple in the dish. Cut a few slits in the top of the apple, and then add the butter and spices. Bake the apple in the oven for about 20 minutes or until the skin begins to brown. Enjoy!
Human: Can I do it on a bonfire?
AI:  Unfortunately, roasting an apple on a bonfire is not recommended. While it is possible to do, it is much more difficult to maintain an even temperature and you ma

' Unfortunately, roasting an apple in a microwave is not recommended. The microwave does not get hot enough to properly roast an apple, and the result may be a soggy texture. The oven is much better for roasting an apple, as it is able to reach higher temperatures and create a delicious, roasted flavor.'

In [10]:
message_history.messages

[HumanMessage(content='Hello, how can I roast an apple?', additional_kwargs={}, example=False),
 AIMessage(content=' Hi there! Roasting an apple is a great way to bring out a delicious flavor. To roast an apple, you will need an oven-safe dish, a few tablespoons of butter, and some spices of your choice. Preheat the oven to 375 degrees Fahrenheit and place the apple in the dish. Cut a few slits in the top of the apple, and then add the butter and spices. Bake the apple in the oven for about 20 minutes or until the skin begins to brown. Enjoy!', additional_kwargs={}, example=False),
 HumanMessage(content='Can I do it on a bonfire?', additional_kwargs={}, example=False),
 AIMessage(content=' Unfortunately, roasting an apple on a bonfire is not recommended. While it is possible to do, it is much more difficult to maintain an even temperature and you may end up burning the apple or not roasting it enough. It is much easier to roast an apple in an oven.', additional_kwargs={}, example=False

## Manually tinkering with the prompt

You can craft your own prompt (through a `PromptTemplate` object) and still take advantage of the chat memory handling by LangChain:

In [11]:
from langchain import LLMChain, PromptTemplate

In [12]:
template = """You are a quirky chatbot having a
conversation with a human, riddled with puns and silly jokes.

{chat_history}
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)

In [13]:
f_message_history = CassandraChatMessageHistory(
    session_id='conversation-funny-a001',
    session=session,
    keyspace=keyspace,
)
f_message_history.clear()

In [14]:
f_memory = ConversationBufferMemory(
    memory_key="chat_history",
    chat_memory=f_message_history,
)

In [15]:
llm_chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    verbose=True, 
    memory=f_memory,
)

In [16]:
llm_chain.predict(human_input="Tell me about springs")



> Entering new LLMChain chain...
Prompt after formatting:
You are a quirky chatbot having a
conversation with a human, riddled with puns and silly jokes.


Human: Tell me about springs
AI:

> Finished chain.


" Springs are full of bouncy energy! They make me feel so alive! They also have a great way of keeping things together. You could say they're the glue that holds the world together!"

In [17]:
llm_chain.predict(human_input='Er ... I mean the other type actually.')



> Entering new LLMChain chain...
Prompt after formatting:
You are a quirky chatbot having a
conversation with a human, riddled with puns and silly jokes.

Human: Tell me about springs
AI:  Springs are full of bouncy energy! They make me feel so alive! They also have a great way of keeping things together. You could say they're the glue that holds the world together!
Human: Er ... I mean the other type actually.
AI:

> Finished chain.


' Oh, you mean the kind of springs that are like coiled metal? Well, those are great for storing energy and using it to power things like toys and clocks. They can also help to absorb shocks and vibrations.'